## Σε αυτό το σημείο της εργασίες πρέπει να συγκρίνουμε τα 3 pipelines που χρησιμοποιήσαμε στο ερώτημα 1Β μεταξύ τους

### Βήματα 
- Διαβάζουμε τα 3 ανακατσκευασμένα κείμενα
- Κάνουμε σύγκριση

In [8]:
original_text=""
pipeline1=""
pipeline2=""
pipeline3=""

In [9]:
try:
    with open('../text_files/text2.txt', 'r', encoding='utf-8') as file:
        original_text = file.read()
    print("File content successfully loaded into original_text.")
except FileNotFoundError:
    print("Error: The file 'text2.txt' was not found. Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into original_text.


In [10]:
try:
    with open("../text_files/first_pipeline_text2.txt", 'r', encoding='utf-8') as file:
        pipeline1 = file.read()
    print("File content successfully loaded into pipeline1.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline1.


In [11]:
try:
    with open("../text_files/second_pipeline_text2.txt", 'r', encoding='utf-8') as file:
        pipeline2 = file.read()
    print("File content successfully loaded into pipeline2.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline2.


In [12]:
try:
    with open("../text_files/third_pipeline_text2.txt", 'r', encoding='utf-8') as file:
        pipeline3 = file.read()
    print("File content successfully loaded into pipeline3.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline3.


## Αρχικά θα συγκρίνουμε το πόσο διαφέρει η κάθε ανακατασκευή από το πρωτότυπο κείμενο με μέθοδο cosine similarity

In [13]:
from collections import Counter
import numpy as np
def cosine_similarity_logic(original_text,pipeline):
    sentences = [original_text, pipeline]
    tokenized = [s.lower().split() for s in sentences]
    vocab = sorted(set(word for doc in tokenized for word in doc))

    tf_matrix = []
    for doc in tokenized:
        word_counts = Counter(doc)
        tf_vector = [word_counts[word] / len(doc) for word in vocab]
        tf_matrix.append(tf_vector)
        
        
    N = len(tokenized) 
    df_vector = [sum(1 for doc in tokenized if word in doc) for word in vocab]
    idf_vector = [np.log((N + 1) / (df + 1)) + 1 for df in df_vector]
    tfidf_matrix = np.array(tf_matrix) * np.array(idf_vector)
    
    
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    print(f"Cosine Similarity: {similarity}")
    
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [14]:
cosine_similarity_logic(original_text,pipeline1)
cosine_similarity_logic(original_text,pipeline2)
cosine_similarity_logic(original_text,pipeline3)

Cosine Similarity: 0.9032013745350934
Cosine Similarity: 0.9100695080697159
Cosine Similarity: 0.6461714378179904


## Ground truth κείμενο από το gemini

In [15]:
gemini="""During our last discussion, I mentioned the new submission we've been awaiting since last autumn. The updates were a bit unclear, as they didn't seem to include full reviewer or editor feedback.

Despite some recent delays and less frequent communication, I believe the team truly put in their best effort for the paper and our collaboration. We should all be grateful for the acceptance and the hard work that led to the Springer link finally appearing last week.

Could you please remind me if the doctor still plans to edit the acknowledgments section before resubmitting? I haven't seen that part finalized yet, and I apologize if I missed it.

Let's ensure everyone is safe and celebrate this outcome with some strong coffee as we look to future goals!"""

In [16]:
cosine_similarity_logic(gemini,pipeline1)
cosine_similarity_logic(gemini,pipeline2)
cosine_similarity_logic(gemini,pipeline3)

Cosine Similarity: 0.6292970464242968
Cosine Similarity: 0.6435868454573861
Cosine Similarity: 0.5042018246913165


## Ground truth κείμενο από το deepseek

In [17]:
deepseek="""During our final discussion, I mentioned the new submission—the one we’ve been waiting for since last autumn. However, the updates were a bit confusing, as they didn’t seem to include the reviewers’ or editor’s full feedback.

That said, I truly believe the team, despite recent delays and limited communication, did their best for the paper and our collaboration. We should all be grateful for the acceptance and their efforts, especially now that the Springer link is finally available (as of last week, I believe).

Also, could you kindly confirm whether the doctor still plans to edit the acknowledgments section before resubmitting? I haven’t seen the final version of that part yet—or perhaps I missed it. If so, I apologize.

In any case, let’s ensure everything is in order, celebrate this achievement with strong coffee, and focus on our next goals!"""

In [18]:
cosine_similarity_logic(deepseek,pipeline1)
cosine_similarity_logic(deepseek,pipeline2)
cosine_similarity_logic(deepseek,pipeline3)

Cosine Similarity: 0.6382258047137673
Cosine Similarity: 0.6592709455115673
Cosine Similarity: 0.49522647560750943


## Ground truth κείμενο από το chat gpt

In [19]:
chat_gpt="""During our final discussion, I mentioned the new submission — the one we have been waiting for since last autumn. However, the updates were confusing because they did not include the full feedback from the reviewer or possibly the editor.

Nevertheless, I believe the team, despite some delays and reduced communication in recent days, has truly done their best for the paper and the collaboration. We should all be grateful for the acceptance and the hard work, especially now that the Springer link finally came out last week, I believe.

Also, please kindly remind me if the doctor still plans to edit the acknowledgments section before sending it again. I haven’t seen the final version of that part yet, but I may have missed it—apologies if so.

Overall, let’s make sure everyone stays safe and celebrate this achievement with strong coffee and new goals ahead."""

In [20]:
cosine_similarity_logic(chat_gpt,pipeline1)
cosine_similarity_logic(chat_gpt,pipeline2)
cosine_similarity_logic(chat_gpt,pipeline3)

Cosine Similarity: 0.7355473565481143
Cosine Similarity: 0.7490192052576788
Cosine Similarity: 0.5815827923398523
